In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import one_hot
from sklearn import preprocessing

In [2]:
player_df = pd.read_csv("all_seasons.csv")

In [3]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11145 entries, 0 to 11144
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         11145 non-null  int64  
 1   player_name        11145 non-null  object 
 2   Position           10522 non-null  object 
 3   team_abbreviation  11145 non-null  object 
 4   age                11145 non-null  int64  
 5   player_height      11145 non-null  float64
 6   player_weight      11145 non-null  float64
 7   college            11145 non-null  object 
 8   country            11145 non-null  object 
 9   draft_year         11145 non-null  object 
 10  draft_round        11145 non-null  object 
 11  draft_number       11145 non-null  object 
 12  gp                 11145 non-null  int64  
 13  pts                11145 non-null  float64
 14  reb                11145 non-null  float64
 15  ast                11145 non-null  float64
 16  net_rating         111

In [4]:
player_df = player_df.drop(player_df.loc[player_df["years_played"] == "#VALUE!"].index)
player_df = player_df.drop(player_df.loc[player_df["draft_year"] == "Undrafted"].index)
player_df = player_df.drop(player_df.loc[player_df["draft_round"] == "Undrafted"].index)
player_df = player_df.drop(player_df.loc[player_df["draft_number"] == "Undrafted"].index)
player_df.head()

,Unnamed: 0,player_name,Position,team_abbreviation,age,player_height,player_weight,college,country,draft_year,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,years_played
0,0,Dennis Rodman,F,CHI,36,198.12,99.790240,Southeastern Oklahoma State,USA,1986,...,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996,10
1,1,Dwayne Schintzius,C,LAC,28,215.90,117.933920,Florida,USA,1990,...,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996,6
2,2,Earl Cureton,F-C,TOR,39,205.74,95.254320,Detroit Mercy,USA,1979,...,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996,17
3,3,Ed O'Bannon,F,DAL,24,203.20,100.697424,UCLA,USA,1995,...,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996,1
4,4,Ed Pinckney,F,MIA,34,205.74,108.862080,Villanova,USA,1985,...,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996,11


In [5]:
player_df["years_played"] = player_df["years_played"].astype(int)
player_df["draft_year"] = player_df["draft_year"].astype(int)
player_df["draft_round"] = player_df["draft_round"].astype(int)
player_df["draft_number"] = player_df["draft_number"].astype(int)

In [6]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9186 entries, 0 to 11142
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         9186 non-null   int64  
 1   player_name        9186 non-null   object 
 2   Position           8778 non-null   object 
 3   team_abbreviation  9186 non-null   object 
 4   age                9186 non-null   int64  
 5   player_height      9186 non-null   float64
 6   player_weight      9186 non-null   float64
 7   college            9186 non-null   object 
 8   country            9186 non-null   object 
 9   draft_year         9186 non-null   int32  
 10  draft_round        9186 non-null   int32  
 11  draft_number       9186 non-null   int32  
 12  gp                 9186 non-null   int64  
 13  pts                9186 non-null   float64
 14  reb                9186 non-null   float64
 15  ast                9186 non-null   float64
 16  net_rating         9186

In [7]:
player_df["avg_reb"] = 0.0
player_df["avg_pts"] = 0.0
player_df["avg_ast"] = 0.0
player_df["last_reb"] = 0.0
player_df["last_ast"] = 0.0
player_df["last_pts"] = 0.0
player_df.head()

,Unnamed: 0,player_name,Position,team_abbreviation,age,player_height,player_weight,college,country,draft_year,...,ts_pct,ast_pct,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
0,0,Dennis Rodman,F,CHI,36,198.12,99.790240,Southeastern Oklahoma State,USA,1986,...,0.479,0.113,1996,10,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Dwayne Schintzius,C,LAC,28,215.90,117.933920,Florida,USA,1990,...,0.430,0.048,1996,6,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Earl Cureton,F-C,TOR,39,205.74,95.254320,Detroit Mercy,USA,1979,...,0.376,0.148,1996,17,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Ed O'Bannon,F,DAL,24,203.20,100.697424,UCLA,USA,1995,...,0.399,0.077,1996,1,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Ed Pinckney,F,MIA,34,205.74,108.862080,Villanova,USA,1985,...,0.611,0.040,1996,11,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
player_name = np.unique(player_df["player_name"])
history = {}
pts_history = {}
ast_history = {}

for name in player_name:
    seasons = player_df.loc[player_df["player_name"] == name]
    history.update({name:seasons["reb"]})

for name in player_name:
    seasons = player_df.loc[player_df["player_name"] == name]
    pts_history.update({name:seasons["pts"]})
    
for name in player_name:
    seasons = player_df.loc[player_df["player_name"] == name]
    ast_history.update({name:seasons["ast"]})

In [9]:
history

{'A.C. Green': 138     7.9
 800     8.1
 1054    4.6
 1319    5.9
 1948    3.8
 Name: reb, dtype: float64,
 'A.J. Bramlett': 1538    2.8
 Name: reb, dtype: float64,
 'A.J. Guyton': 1759    1.1
 2497    1.0
 2852    0.0
 Name: reb, dtype: float64,
 'AJ Hammons': 9147    1.6
 Name: reb, dtype: float64,
 'AJ Price': 5805    1.6
 6490    1.4
 7135    1.4
 7426    2.0
 7730    0.4
 8198    1.2
 Name: reb, dtype: float64,
 'Aaron Brooks': 5025    1.1
 5504    2.0
 5804    2.6
 6491    1.3
 7158    1.5
 7731    1.9
 8199    2.0
 8749    1.5
 9146    1.1
 9647    0.5
 Name: reb, dtype: float64,
 'Aaron Gordon': 8200     3.6
 8600     6.5
 9145     5.1
 9648     7.9
 10628    7.4
 10791    7.6
 Name: reb, dtype: float64,
 'Aaron Gray': 5026    2.8
 5505    3.9
 5818    3.3
 6492    4.2
 7134    5.7
 7571    3.2
 7732    3.0
 Name: reb, dtype: float64,
 'Aaron Holiday': 10627    1.3
 10792    2.3
 Name: reb, dtype: float64,
 'Aaron McKie': 139     2.7
 549     2.9
 1055    2.8
 1682    3.0
 1899

In [10]:
for player in history:
    print(player)
    count = 0
    total_rebounds = 0.0
    for value in history[player]:
        if count == 0:
            total_rebounds = total_rebounds + value
            player_df.at[history[player].index[0], 'last_reb'] = 0
            player_df.at[history[player].index[0], 'avg_reb'] = value
        elif count != 0:
            total_rebounds = total_rebounds + value
            print(total_rebounds)
            player_df.at[history[player].index[count], 'last_reb'] = history[player].values[count-1]
            player_df.at[history[player].index[count], 'avg_reb'] = total_rebounds/(count+1)
        count = count + 1
            

A.C. Green
16.0
20.6
26.5
30.3
A.J. Bramlett
A.J. Guyton
2.1
2.1
AJ Hammons
AJ Price
3.0
4.4
6.4
6.800000000000001
8.0
Aaron Brooks
3.1
5.7
7.0
8.5
10.4
12.4
13.9
15.0
15.5
Aaron Gordon
10.1
15.2
23.1
30.5
38.1
Aaron Gray
6.699999999999999
10.0
14.2
19.9
23.099999999999998
26.099999999999998
Aaron Holiday
3.5999999999999996
Aaron McKie
5.6
8.399999999999999
11.399999999999999
15.499999999999998
19.5
23.9
27.299999999999997
29.799999999999997
31.199999999999996
32.99999999999999
Abdel Nader
3.4
5.3
Acie Earl
Acie Law
2.1
2.8
4.0
Adam Keefe
9.0
12.2
14.399999999999999
17.5
Adam Morrison
4.4
5.4
Admiral Schofield
Adonal Foyle
7.7
13.3
20.3
25.200000000000003
31.200000000000003
35.0
40.5
46.0
48.6
51.1
53.7
Adonis Jordan
Adreian Payne
7.199999999999999
9.0
10.8
Al Harrington
5.1
10.0
16.3
22.5
28.9
35.9
42.8
49.199999999999996
54.599999999999994
60.8
66.39999999999999
70.89999999999999
76.99999999999999
79.69999999999999
82.1
Al Horford
19.0
28.9
38.2
45.2
55.400000000000006
63.80000000000

28.7
29.9
Brian Cardinal
2.3
3.3
7.5
11.4
12.9
15.0
17.6
19.8
20.8
21.900000000000002
22.700000000000003
Brian Cook
5.9
9.3
12.600000000000001
14.700000000000001
15.8
16.400000000000002
18.8
20.8
Brian Evans
2.5
3.7
Brian Grant
15.0
24.8
30.3
39.1
47.1
57.3
64.2
67.9
70.60000000000001
Brian Oliver
Brian Scalabrine
4.2
6.7
11.2
12.799999999999999
14.7
16.3
17.6
18.5
18.9
19.7
Brian Shaw
6.1
7.1
10.0
13.8
15.700000000000001
17.400000000000002
Brian Skinner
8.6
12.899999999999999
17.2
22.0
29.3
35.0
38.5
44.2
47.800000000000004
51.800000000000004
53.50000000000001
53.50000000000001
53.50000000000001
Brice Johnson
2.7
Brook Lopez
16.7
22.7
26.3
33.2
39.2
46.6
54.4
59.8
63.8
68.7
73.2
Brooks Thompson
1.9
Bruce Brown
7.2
Bruno Caboclo
0.6
1.7000000000000002
3.5
8.1
10.0
Bruno Fernando
Bruno Sundov
0.9
3.0
4.0
5.1
7.1
7.699999999999999
Bryant Reeves
16.0
21.5
27.2
33.2
Bryant Stith
6.300000000000001
8.600000000000001
10.500000000000002
14.100000000000001
15.8
Bryce Drew
2.3
3.6999999999999997

41.6
44.300000000000004
Danny Green
2.8
6.3
9.4
12.8
17.0
20.8
24.1
27.700000000000003
31.700000000000003
35.1
Danny Manning
11.7
16.1
19.0
21.6
24.200000000000003
25.6
Dante Cunningham
6.1
9.899999999999999
14.999999999999998
19.099999999999998
22.999999999999996
25.999999999999996
30.199999999999996
34.3
37.199999999999996
Dante Exum
3.6
5.5
7.1
9.0
Darington Hobson
Dario Saric
13.0
18.6
24.5
Darius Bazley
Darius Garland
Darius Johnson-Odom
1.7
Darius Miles
11.4
16.8
21.3
26.0
30.6
32.300000000000004
Darius Miller
2.7
2.9000000000000004
4.9
6.800000000000001
Darius Morris
2.0
3.0
3.7
Darius Songaila
7.300000000000001
11.3
14.9
18.3
21.2
24.3
25.3
Darko Milicic
2.5
5.2
10.7
16.799999999999997
21.099999999999998
25.799999999999997
30.999999999999996
34.3
35.3
Darnell Jackson
2.5
4.1
Darrell Arthur
8.0
12.3
15.200000000000001
18.3
21.2
25.4
28.099999999999998
28.9
Darren Collison
5.3
8.4
11.100000000000001
13.500000000000002
16.700000000000003
19.000000000000004
21.200000000000003
23.80

4.1000000000000005
Elmer Bennett
Elmore Spencer
Elton Brand
20.1
31.700000000000003
43.0
53.3
62.8
72.8
82.1
90.1
98.89999999999999
104.99999999999999
113.29999999999998
120.49999999999999
126.49999999999999
131.39999999999998
134.2
137.89999999999998
Emeka Okafor
20.9
32.2
42.900000000000006
53.00000000000001
62.00000000000001
71.5
79.4
88.2
92.8
Emmanuel Mudiay
6.6
9.0
12.3
14.700000000000001
Enes Kanter
8.5
16.0
24.9
33.0
39.7
50.7
60.5
68.4
Ennis Whatley
Eric Bledsoe
4.4
7.4
12.100000000000001
17.3
21.3
26.1
29.900000000000002
34.5
39.2
Eric Gordon
5.2
8.1
10.899999999999999
12.7
15.299999999999999
17.9
20.099999999999998
22.799999999999997
25.299999999999997
27.499999999999996
29.399999999999995
Eric Leckner
Eric Maynor
3.1
4.5
5.2
6.5
Eric Mobley
Eric Montross
10.7
13.7
15.1
18.3
21.2
Eric Murdock
2.8
5.1
7.0
Eric Paschall
Eric Piatkowski
5.1
8.0
11.0
14.0
16.6
19.1
20.6
21.8
22.6
23.400000000000002
24.200000000000003
Eric Riley
6.199999999999999
Eric Snow
2.3
5.699999999999999
8

21.7
26.799999999999997
32.5
35.0
38.3
Jason Richardson
8.899999999999999
15.599999999999998
21.5
27.3
32.4
37.8
42.199999999999996
47.3
51.4
55.0
58.8
62.3
Jason Sasser
2.2
Jason Smith
5.4
8.5
13.4
17.0
22.8
26.8
29.7
33.2
34.800000000000004
37.400000000000006
Jason Terry
5.3
8.8
12.200000000000001
16.3
18.7
20.7
23.599999999999998
26.099999999999998
28.499999999999996
30.299999999999997
32.199999999999996
34.599999999999994
36.599999999999994
37.699999999999996
39.3
40.4
41.8
42.699999999999996
Jason Thompson
15.9
22.0
28.9
35.6
42.0
48.5
51.4
Jason Williams
5.9
8.3
11.3
14.100000000000001
16.1
17.8
20.2
22.5
24.4
25.9
27.0
Javaris Crittenton
4.800000000000001
Jawun Evans
3.3
Jaxson Hayes
Jay Williams
Jaylen Brown
7.7
11.9
18.3
Jaylen Nowell
Jayson Tatum
11.0
18.1
Jayson Williams
27.1
39.1
Jeff Ayres
4.2
7.0
10.5
12.8
14.100000000000001
Jeff Foster
7.2
14.0
17.6
25.0
34.0
43.1
51.2
59.900000000000006
66.80000000000001
71.9
78.2
82.0
Jeff Grayer
2.3
Jeff Green
11.4
17.4
22.2
26.099999

50.7
52.300000000000004
Kevon Looney
4.3
7.6
12.8
16.1
Keyon Dooling
1.4
2.7
4.1
5.3
6.9
8.200000000000001
9.600000000000001
11.600000000000001
12.600000000000001
14.100000000000001
14.900000000000002
15.000000000000002
Khalid El-Amin
Khalid Reeves
4.199999999999999
4.799999999999999
6.099999999999999
Khris Middleton
5.699999999999999
10.1
13.899999999999999
18.099999999999998
23.299999999999997
29.299999999999997
35.599999999999994
Khyri Thomas
0.8
Kim English
Kirk Haston
1.9
Kirk Hinrich
7.3
10.9
14.3
17.6
20.0
23.5
26.0
28.1
30.700000000000003
33.300000000000004
35.1
36.6
Kirk Snyder
4.2
6.300000000000001
9.8
Klay Thompson
6.1
9.2
12.399999999999999
16.2
19.9
23.7
27.5
Kobe Bryant
5.0
10.3
16.6
22.5
28.0
34.9
40.4
46.3
51.599999999999994
57.3
63.599999999999994
68.8
74.2
79.3
84.7
90.3
94.6
100.3
104.0
Korleone Young
Kosta Koufos
4.2
6.800000000000001
12.200000000000001
19.1
24.3
29.6
35.0
40.7
47.300000000000004
51.50000000000001
Kosta Perovic
Kostas Antetokounmpo
0.8
Kostas Papani

5.699999999999999
9.799999999999999
14.7
16.5
19.3
21.400000000000002
23.900000000000002
24.400000000000002
Melvin Frazier Jr.
0.8
Metta World Peace
8.2
13.1
18.3
23.6
30.0
35.1
41.6
47.4
52.6
56.9
60.199999999999996
63.599999999999994
68.6
70.6
73.1
73.89999999999999
Meyers Leonard
6.5
11.0
16.1
19.3
21.400000000000002
25.200000000000003
30.300000000000004
Mfiondu Kabengele
Michael Beasley
11.8
17.4
21.799999999999997
25.599999999999998
28.7
32.4
37.3
40.699999999999996
46.3
48.599999999999994
Michael Bradley
7.0
8.4
10.0
12.3
Michael Cage
7.8
11.899999999999999
Michael Carter-Williams
11.5
16.6
20.0
22.7
25.2
28.599999999999998
Michael Dickerson
5.1
8.399999999999999
11.399999999999999
12.399999999999999
Michael Doleac
7.1
10.6
14.6
17.5
21.2
24.4
27.099999999999998
29.9
31.9
Michael Finley
9.8
15.100000000000001
21.400000000000002
26.6
31.8
37.6
42.1
46.2
49.400000000000006
52.10000000000001
55.20000000000001
58.50000000000001
60.00000000000001
Michael Gbinije
Michael Jordan
11.7
17

24.099999999999998
24.9
27.5
28.7
Raul Lopez
3.2
Raul Neto
2.3
3.5
5.2
6.300000000000001
Ray Allen
8.9
13.100000000000001
17.5
22.7
27.2
32.2
37.300000000000004
41.7
46.0
50.5
54.2
57.7
60.900000000000006
64.30000000000001
67.4
70.10000000000001
72.9
Ray McCallum
4.4
5.5
Ray Spalding
Raymond Felton
6.699999999999999
9.7
13.5
17.1
20.700000000000003
23.200000000000003
26.1
29.1
30.0
33.2
35.900000000000006
37.800000000000004
38.800000000000004
Reece Gaines
1.7
1.7
Reggie Bullock
2.7
4.5
6.6
9.1
11.8
14.200000000000001
Reggie Geary
1.5
Reggie Jackson
3.5999999999999996
7.5
11.7
14.899999999999999
17.099999999999998
19.9
22.5
25.5
Reggie Miller
6.4
9.100000000000001
12.100000000000001
15.600000000000001
18.400000000000002
20.900000000000002
23.3
25.7
Reggie Williams
Renaldo Balkman
7.6
11.399999999999999
13.2
14.1
16.0
Rex Chapman
5.3
8.0
9.5
Rex Walters
2.4
3.9
5.0
Richard Hamilton
4.9
8.3
12.200000000000001
15.8
19.7
22.9
26.7
30.0
33.1
35.800000000000004
38.1
40.5
42.2
Richard Jefferso

53.9
Steven Hunter
4.6
7.5
10.5
14.4
19.2
20.7
22.7
Steven Smith
7.5
11.7
15.5
18.9
21.4
23.299999999999997
24.4
25.7
Stromile Swift
9.9
15.600000000000001
20.5
25.1
29.5
34.1
37.7
40.1
Sun Sun
Svi Mykhailiuk
2.7
T.J. Ford
7.5
10.6
12.6
16.1
19.3
21.3
22.6
T.J. Warren
9.1
13.0
TJ Leaf
3.7
6.300000000000001
TJ Warren
5.2
10.3
Taj Gibson
13.2
18.5
23.8
30.6
37.0
43.9
50.1
57.2
63.7
68.10000000000001
Talen Horton-Tucker
Tamar Slay
2.0
3.8
Tariq Abdul-Wahad
5.8
10.6
12.6
16.1
19.0
Taurean Green
Taurean Prince
7.4
11.0
16.9
Taylor Griffin
Tayshaun Prince
5.9
11.2
15.399999999999999
20.599999999999998
25.5
31.3
36.4
40.6
45.1
49.5
52.6
56.2
58.1
Terance Mann
Terence Morris
5.7
7.4
Terrance Ferguson
2.7
4.1
Terrell Brandon
7.4
11.100000000000001
14.500000000000002
18.3
21.2
Terrence Jones
10.3
17.0
21.2
26.9
28.9
Terrence Ross
5.1
7.8999999999999995
10.399999999999999
12.999999999999998
15.999999999999998
19.5
22.7
Terrence Williams
6.9
10.3
12.100000000000001
Terry Cummings
7.899999999999999

35.8
45.199999999999996
56.0
65.9
71.30000000000001
Yaroslav Korolev
0.8
Yi Jianlian
10.5
17.7
21.599999999999998
23.2
Yinka Dare
3.7
Zach Collins
7.5
11.5
Zach LaVine
5.6
9.0
12.9
17.6
22.400000000000002
Zach Randolph
6.2
16.7
26.299999999999997
34.3
44.4
54.7
64.8
76.5
88.7
96.7
107.9
118.0
128.5
136.3
144.5
151.2
Zan Tabak
7.5
9.3
13.200000000000001
Zarko Cabarkapa
4.6
6.3999999999999995
Zaza Pachulia
8.0
15.9
22.8
26.8
32.5
36.3
40.5
48.4
54.9
61.199999999999996
68.0
77.4
83.30000000000001
88.00000000000001
91.90000000000002
Zeljko Rebraca
7.0
9.4
12.600000000000001
14.8
Zhaire Smith
2.5
Zhou Qi
1.2
Zion Williamson
Zoran Planinic
2.7
4.0
Zydrunas Ilgauskas
17.6
24.3
29.700000000000003
37.2
45.300000000000004
53.900000000000006
61.50000000000001
69.2
78.5
86.0
91.4
95.4


In [11]:
for player in pts_history:
    print(player)
    count = 0
    total_points = 0.0
    for value in pts_history[player]:
        if count == 0:
            total_points = total_points + value
            print(pts_history[player].index[count])
            player_df.at[pts_history[player].index[0], 'last_pts'] = 0
            player_df.at[pts_history[player].index[0], 'avg_pts'] = value
        elif count != 0:
            total_points = total_points + value
            print("loop two")
            print(pts_history[player].index[count])
            print(pts_history[player].values[count - 1])
            player_df.at[pts_history[player].index[count], 'last_pts'] = pts_history[player].values[count-1]
            player_df.at[pts_history[player].index[count], 'avg_pts'] = total_points/(count+1)
        count = count + 1

A.C. Green
138
loop two
800
7.2
loop two
1054
7.3
loop two
1319
4.9
loop two
1948
5.0
A.J. Bramlett
1538
A.J. Guyton
1759
loop two
2497
6.0
loop two
2852
5.4
AJ Hammons
9147
AJ Price
5805
loop two
6490
7.3
loop two
7135
6.5
loop two
7426
3.9
loop two
7730
7.7
loop two
8198
1.6
Aaron Brooks
5025
loop two
5504
5.2
loop two
5804
11.2
loop two
6491
19.6
loop two
7158
10.7
loop two
7731
7.1
loop two
8199
9.0
loop two
8749
11.6
loop two
9146
7.1
loop two
9647
5.0
Aaron Gordon
8200
loop two
8600
5.2
loop two
9145
9.2
loop two
9648
12.7
loop two
10628
17.6
loop two
10791
16.0
Aaron Gray
5026
loop two
5505
4.3
loop two
5818
3.5
loop two
6492
3.3
loop two
7134
3.1
loop two
7571
3.9
loop two
7732
2.8
Aaron Holiday
10627
loop two
10792
5.9
Aaron McKie
139
loop two
549
5.2
loop two
1055
4.1
loop two
1682
4.8
loop two
1899
8.0
loop two
2496
11.6
loop two
2928
12.2
loop two
3366
9.0
loop two
3508
9.2
loop two
4282
2.2
loop two
4576
0.5
Abdel Nader
9651
loop two
10626
3.0
loop two
10793
4.0
Acie Earl


loop two
5792
5.3
Anthony Mason
155
loop two
747
16.2
loop two
1477
12.8
loop two
1918
11.6
loop two
2476
16.1
loop two
2694
9.6
Anthony Miller
156
loop two
746
0.0
loop two
1029
2.1
loop two
1478
2.4
loop two
1919
3.7
loop two
3827
0.5
Anthony Parker
745
loop two
1030
1.9
loop two
1479
1.0
loop two
4551
3.6
loop two
5004
12.4
loop two
5502
12.5
loop two
5793
10.7
loop two
6480
7.3
loop two
7051
8.3
Anthony Peeler
157
loop two
744
14.5
loop two
1031
12.3
loop two
1480
9.6
loop two
1920
9.8
loop two
2475
10.5
loop two
2704
9.0
loop two
3344
7.7
loop two
3828
5.7
Anthony Randolph
5500
loop two
5794
7.9
loop two
6481
11.6
loop two
7049
7.6
loop two
7558
7.4
loop two
8061
3.7
Antoine Carr
158
loop two
743
7.4
loop two
1032
5.7
loop two
1456
2.6
Antoine Walker
159
loop two
742
17.5
loop two
1033
22.4
loop two
1455
18.7
loop two
1921
20.5
loop two
2474
23.4
loop two
2718
22.1
loop two
3343
20.1
loop two
3854
14.0
loop two
4306
19.1
loop two
4553
12.2
loop two
5005
8.5
Antoine Wright
4305
loo

4635
7.9
loop two
5086
6.9
loop two
5497
4.7
loop two
5957
2.5
loop two
6544
1.5
loop two
7085
4.8
Brian Evans
131
loop two
791
1.4
loop two
885
4.5
Brian Grant
132
loop two
783
10.5
loop two
888
12.1
loop two
1523
11.5
loop two
1889
7.3
loop two
2506
15.2
loop two
2720
9.3
loop two
3374
10.3
loop two
3768
8.7
loop two
4218
3.8
Brian Oliver
773
Brian Scalabrine
2505
loop two
2693
2.1
loop two
3373
3.1
loop two
3766
3.5
loop two
4217
6.3
loop two
4636
2.9
loop two
5087
4.0
loop two
5498
1.8
loop two
5970
3.5
loop two
6525
1.5
loop two
7084
1.1
Brian Shaw
133
loop two
767
7.2
loop two
883
6.3
loop two
1484
0.0
loop two
1890
4.1
loop two
2504
5.3
loop two
2743
2.9
Brian Skinner
881
loop two
1512
4.1
loop two
1891
5.4
loop two
2503
4.1
loop two
2692
3.4
loop two
3372
6.0
loop two
3765
10.5
loop two
4216
4.8
loop two
4637
2.9
loop two
5089
4.4
loop two
5499
3.3
loop two
5956
4.2
loop two
6524
1.6
loop two
7083
0.0
Brice Johnson
9559
loop two
9633
1.3
Brook Lopez
5501
loop two
5954
13.0
loop

7.7
loop two
3293
5.9
loop two
3746
6.3
loop two
4253
9.8
loop two
5055
10.2
loop two
5587
3.6
Chris Mills
213
loop two
690
13.4
loop two
958
9.7
loop two
1509
10.3
loop two
1971
16.1
loop two
2423
12.0
loop two
2641
7.4
Chris Morris
214
loop two
691
4.3
loop two
959
4.3
Chris Mullin
215
loop two
692
14.5
loop two
960
11.3
loop two
1508
10.1
loop two
1972
5.1
Chris Owens
2651
Chris Paul
4252
loop two
4656
16.1
loop two
5056
17.3
loop two
5588
21.1
loop two
5995
22.8
loop two
6420
18.7
loop two
6760
15.9
loop two
7551
19.8
loop two
7853
16.9
loop two
8229
19.1
loop two
8957
19.1
loop two
9469
19.5
loop two
9721
18.1
loop two
10294
18.6
loop two
10862
15.6
Chris Porter
1973
Chris Richard
5058
loop two
5993
1.9
Chris Robinson
216
loop two
678
4.6
Chris Singleton
6759
loop two
7549
4.6
loop two
7852
4.1
Chris Taft
4251
Chris Webber
217
loop two
677
20.1
loop two
961
21.9
loop two
1485
20.0
loop two
1974
24.5
loop two
2422
27.1
loop two
2664
24.5
loop two
3292
23.0
loop two
3745
18.7
loop t

7483
10.4
loop two
7810
12.0
loop two
8130
11.4
loop two
8885
16.1
loop two
9451
14.0
loop two
9712
13.2
loop two
10354
12.4
Darrin Hancock
184
Darrun Hilliard
8884
loop two
9473
4.0
loop two
9695
3.3
David Andersen
5902
loop two
6432
5.8
David Harrison
3773
loop two
4316
6.1
loop two
4627
5.7
loop two
4924
3.0
David Lee
4399
loop two
4628
5.1
loop two
4925
10.7
loop two
5448
10.8
loop two
5901
16.0
loop two
6433
20.2
loop two
6785
16.5
loop two
7484
20.1
loop two
7811
18.5
loop two
8131
18.2
loop two
8883
7.9
loop two
9474
7.8
David Noel
4629
David Robinson
186
loop two
721
17.7
loop two
942
21.6
loop two
1413
15.8
loop two
1869
17.8
loop two
2449
14.4
loop two
2745
12.2
David Vaughn
187
loop two
728
2.3
loop two
943
4.1
David West
3316
loop two
3771
3.8
loop two
4397
6.2
loop two
4631
17.1
loop two
4926
18.3
loop two
5447
20.6
loop two
5900
21.0
loop two
6434
19.0
loop two
6784
18.9
loop two
7485
12.8
loop two
7827
17.1
loop two
8134
14.0
loop two
8895
11.7
loop two
9476
7.1
loop two

...

loop two
6059
13.3
loop two
6615
12.6
Perry Jones III
7257
loop two
7701
2.3
loop two
8569
3.5
Pervis Ellison
411
loop two
474
2.5
loop two
1719
3.0
loop two
2169
1.8
Pete Chilcutt
415
loop two
473
3.4
loop two
1170
4.9
loop two
1720
3.6
Pete Myers
472
Peter John Ramos
3681
Peyton Siva
7700
Pierre Jackson
9344
Pooh Richardson
333
loop two
471
5.6
loop two
1172
4.2
Popeye Jones
362
loop two
470
7.8
loop two
1173
8.6
loop two
1721
3.0
loop two
2168
2.6
loop two
2232
3.6
loop two
3031
7.0
loop two
3100
2.0
Predrag Drobnjak
2231
loop two
3032
6.8
loop two
3099
9.4
loop two
3625
6.3
Priest Lauderdale
331
loop two
469
3.2
Primoz Brezec
2230
loop two
3034
2.0
loop two
3098
1.9
loop two
3708
1.6
loop two
4165
13.0
loop two
4805
12.4
loop two
5297
5.0
loop two
6035
2.2
Quentin Richardson
2167
loop two
2229
8.1
loop two
3035
13.3
loop two
3097
9.4
loop two
3707
17.2
loop two
4164
14.9
loop two
4774
8.2
loop two
5296
13.0
loop two
5636
8.1
loop two
6036
10.2
loop two
6612
8.9
loop two
6814
4.4
lo

10419
14.7
loop two
10920
11.2
Rodney Rogers
273
loop two
615
13.2
loop two
1131
15.1
loop two
1583
7.4
loop two
2171
13.8
loop two
2370
12.2
loop two
2898
11.9
loop two
3236
7.0
loop two
3712
7.8
Rodney Stuckey
5329
loop two
5629
7.6
loop two
6033
13.4
loop two
6661
16.6
loop two
6893
15.5
loop two
7238
14.8
loop two
7634
11.5
loop two
8265
13.9
loop two
8680
12.6
loop two
9331
8.9
Rodney White
2369
loop two
2899
3.5
loop two
3235
9.0
loop two
3737
7.5
Rodrick Rhodes
614
loop two
1132
5.8
loop two
1584
3.3
Rodrigue Beaubois
6045
loop two
6675
7.1
loop two
6894
8.4
loop two
7239
8.9
Roger Mason Jr.
2900
loop two
3234
1.8
loop two
4689
3.7
loop two
5328
2.7
loop two
5628
9.1
loop two
6032
11.8
loop two
6660
6.3
loop two
6895
2.9
loop two
7240
5.5
loop two
7642
5.3
Roko Ukic
5627
loop two
6017
4.2
Romeo Langford
10919
Ron Harper
249
loop two
613
6.3
loop two
1133
9.3
loop two
1585
11.2
loop two
2172
7.0
Ron Mercer
612
loop two
1134
15.3
loop two
1586
17.0
loop two
2175
16.9
loop two
2368

loop two
10908
16.8
Spencer Hawes
5239
loop two
5745
4.7
loop two
6093
11.4
loop two
6560
10.0
loop two
6865
7.2
loop two
7305
9.6
loop two
7748
11.0
loop two
8496
13.2
loop two
9006
5.8
loop two
9324
6.0
Spud Webb
641
Stacey Augmon
276
loop two
640
4.7
loop two
1049
5.7
loop two
1562
4.3
loop two
2111
3.4
loop two
2397
4.7
loop two
2874
4.7
loop two
3261
3.0
loop two
3682
5.8
loop two
4107
3.5
Stacey King
303
Stanley Johnson
9007
loop two
9325
8.1
loop two
9891
4.4
loop two
10381
8.7
loop two
10909
6.9
Stanley Roberts
277
loop two
639
9.5
loop two
1048
6.2
loop two
1564
2.3
Stephane Lasme
5238
Stephen Curry
6094
loop two
6559
17.5
loop two
6866
18.6
loop two
7313
14.7
loop two
7708
22.9
loop two
8497
24.0
loop two
9008
23.8
loop two
9326
30.1
loop two
9889
25.3
loop two
10382
26.4
loop two
10911
27.3
Stephen Jackson
2110
loop two
2396
8.2
loop two
2875
3.9
loop two
3260
11.8
loop two
3652
18.1
loop two
4105
18.7
loop two
4755
16.4
loop two
5236
15.5
loop two
5743
20.1
loop two
6096
20

loop two
2309
0.6
loop two
2956
2.0
Travis Leslie
6708
Travis Outlaw
3177
loop two
3651
1.0
loop two
4112
5.4
loop two
4731
5.8
loop two
5272
9.6
loop two
5683
13.3
loop two
6062
12.8
loop two
6604
9.1
loop two
6709
9.2
loop two
7289
4.3
loop two
7783
5.3
Tremaine Fowlkes
2308
loop two
2933
3.4
loop two
3176
4.4
loop two
3653
1.2
Tremont Waters
11007
Trenton Hassell
2331
loop two
2957
8.7
loop two
3187
4.2
loop two
3680
5.0
loop two
4113
6.6
loop two
4720
9.2
loop two
5271
6.7
loop two
5682
2.0
loop two
6063
3.7
Trevor Ariza
3654
loop two
4114
5.9
loop two
4721
4.6
loop two
5270
8.9
loop two
5681
5.5
loop two
6064
8.9
loop two
6603
14.9
loop two
6710
11.0
loop two
7290
10.8
loop two
7782
9.5
loop two
8481
14.4
loop two
8977
12.8
loop two
9149
12.7
loop two
9905
11.7
loop two
10480
11.7
loop two
11009
12.5
Trevor Booker
6602
loop two
6711
5.3
loop two
7291
8.4
loop two
7781
5.3
loop two
8482
6.8
loop two
8978
7.2
loop two
9158
5.9
loop two
9904
10.0
Trey Burke
7780
loop two
8483
12.8
lo

10.0
loop two
10972
6.0
Xavier Henry
6583
loop two
6707
4.3
loop two
7175
5.3
loop two
7715
3.9
loop two
8143
10.0
Xavier McDaniel
299
loop two
582
5.6
Yao Ming
2923
loop two
3211
13.5
loop two
3662
17.5
loop two
4122
18.3
loop two
4741
22.3
loop two
5254
25.0
loop two
5700
22.0
loop two
6582
19.7
Yaroslav Korolev
4121
loop two
4742
1.1
Yi Jianlian
5253
loop two
5699
8.6
loop two
6082
8.6
loop two
6581
12.0
loop two
6705
5.6
Yinka Dare
300
loop two
581
1.4
Zach Collins
9865
loop two
10452
4.4
loop two
10975
6.6
Zach LaVine
8141
loop two
9063
10.1
loop two
9132
14.0
loop two
9864
18.9
loop two
10453
16.7
loop two
10976
23.7
Zach Randolph
2338
loop two
2924
2.8
loop two
3210
8.4
loop two
3660
20.1
loop two
4120
18.9
loop two
4743
18.0
loop two
5252
23.6
loop two
5698
17.6
loop two
6083
20.8
loop two
6579
20.8
loop two
6704
20.1
loop two
7174
11.6
loop two
7714
15.4
loop two
8140
17.4
loop two
9062
16.1
loop two
9133
15.3
loop two
9863
14.1
Zan Tabak
301
loop two
580
6.5
loop two
1756
5.4

In [12]:
for player in ast_history:
    print(player)
    count = 0
    total_assists = 0.0
    for value in ast_history[player]:
        if count == 0:
            total_assists = total_assists + value
            print(ast_history[player].index[count])
            player_df.at[ast_history[player].index[0], 'last_ast'] = 0
            player_df.at[ast_history[player].index[0], 'avg_ast'] = value
        elif count != 0:
            total_assists = total_assists + value
            print("loop two")
            print(ast_history[player].index[count])
            print(ast_history[player].values[count - 1])
            player_df.at[ast_history[player].index[count], 'last_ast'] = ast_history[player].values[count-1]
            player_df.at[ast_history[player].index[count], 'avg_ast'] = total_assists/(count+1)
        count = count + 1

A.C. Green
138
loop two
800
0.8
loop two
1054
1.5
loop two
1319
0.5
loop two
1948
1.0
A.J. Bramlett
1538
A.J. Guyton
1759
loop two
2497
1.9
loop two
2852
1.8
AJ Hammons
9147
AJ Price
5805
loop two
6490
1.9
loop two
7135
2.2
loop two
7426
2.0
loop two
7730
3.6
loop two
8198
0.5
Aaron Brooks
5025
loop two
5504
1.7
loop two
5804
3.0
loop two
6491
5.3
loop two
7158
3.9
loop two
7731
2.2
loop two
8199
3.2
loop two
8749
3.2
loop two
9146
2.6
loop two
9647
1.9
Aaron Gordon
8200
loop two
8600
0.7
loop two
9145
1.6
loop two
9648
1.9
loop two
10628
2.3
loop two
10791
3.7
Aaron Gray
5026
loop two
5505
0.7
loop two
5818
0.8
loop two
6492
0.7
loop two
7134
0.4
loop two
7571
0.6
loop two
7732
0.8
Aaron Holiday
10627
loop two
10792
1.7
Aaron McKie
139
loop two
549
1.9
loop two
1055
2.2
loop two
1682
2.0
loop two
1899
2.9
loop two
2496
5.0
loop two
2928
3.7
loop two
3366
3.5
loop two
3508
2.6
loop two
4282
1.5
loop two
4576
0.8
Abdel Nader
9651
loop two
10626
0.5
loop two
10793
0.3
Acie Earl
141
Acie 

1916
3.4
loop two
2696
1.7
loop two
3346
0.3
loop two
3825
1.0
loop two
4310
2.4
Anthony Johnson
748
loop two
1028
4.3
loop two
1476
2.2
loop two
1917
1.3
loop two
2477
1.5
loop two
2695
1.4
loop two
3345
1.3
loop two
3826
2.8
loop two
4308
4.8
loop two
4550
4.3
loop two
5003
3.0
loop two
5503
3.8
loop two
5792
2.5
Anthony Mason
155
loop two
747
5.7
loop two
1477
4.2
loop two
1918
4.5
loop two
2476
3.1
loop two
2694
4.2
Anthony Miller
156
loop two
746
0.0
loop two
1029
0.1
loop two
1478
0.2
loop two
1919
0.5
loop two
3827
0.0
Anthony Parker
745
loop two
1030
0.5
loop two
1479
0.0
loop two
4551
0.6
loop two
5004
2.1
loop two
5502
2.2
loop two
5793
3.4
loop two
6480
1.9
loop two
7051
3.0
Anthony Peeler
157
loop two
744
3.6
loop two
1031
3.6
loop two
1480
2.8
loop two
1920
2.4
loop two
2475
2.6
loop two
2704
2.2
loop two
3344
3.0
loop two
3828
1.6
Anthony Randolph
5500
loop two
5794
0.8
loop two
6481
1.3
loop two
7049
0.8
loop two
7558
0.6
loop two
8061
0.3
Antoine Carr
158
loop two
743
0

loop two
5087
1.1
loop two
5498
0.8
loop two
5970
0.5
loop two
6525
0.5
loop two
7084
0.3
Brian Shaw
133
loop two
767
4.1
loop two
883
4.4
loop two
1484
1.0
loop two
1890
2.7
loop two
2504
3.2
loop two
2743
1.5
Brian Skinner
881
loop two
1512
0.0
loop two
1891
0.3
loop two
2503
0.5
loop two
2692
0.3
loop two
3372
0.2
loop two
3765
0.9
loop two
4216
0.9
loop two
4637
0.5
loop two
5089
0.9
loop two
5499
0.2
loop two
5956
0.5
loop two
6524
0.0
loop two
7083
0.0
Brice Johnson
9559
loop two
9633
0.3
Brook Lopez
5501
loop two
5954
1.0
loop two
6515
2.3
loop two
7082
1.6
loop two
7620
1.2
loop two
7837
0.9
loop two
8263
0.9
loop two
8931
0.7
loop two
9518
2.0
loop two
9634
2.3
loop two
10240
1.7
loop two
10773
1.2
Brooks Thompson
149
loop two
768
2.7
Bruce Brown
10229
loop two
10787
1.2
Bruno Caboclo
8254
loop two
8930
0.0
loop two
9534
0.2
loop two
9635
0.4
loop two
10241
0.3
loop two
10774
1.5
Bruno Fernando
10775
Bruno Sundov
880
loop two
1514
0.3
loop two
1893
0.1
loop two
2473
0.2
loop t

Christian Eyenga
6424
loop two
6756
0.8
Christian Laettner
205
loop two
675
2.7
loop two
963
2.6
loop two
1487
1.5
loop two
1976
2.3
loop two
2420
1.6
loop two
2689
2.6
loop two
3289
3.1
loop two
3743
1.9
Chuck Person
674
loop two
964
1.4
loop two
1488
1.2
Chucky Brown
192
loop two
673
0.4
loop two
965
0.7
loop two
1490
1.2
loop two
1951
1.0
loop two
2442
0.4
Clar. Weatherspoon
177
loop two
672
1.7
loop two
966
1.1
loop two
1491
0.7
loop two
1950
1.2
loop two
2443
1.3
loop two
2687
1.1
loop two
3311
0.9
loop two
3767
0.6
Cleanthony Early
8231
loop two
8845
0.9
Clifford Robinson
166
loop two
671
3.2
loop two
947
2.1
loop two
1492
2.6
loop two
1949
2.8
loop two
2444
2.9
loop two
2686
2.5
loop two
3312
3.3
loop two
3769
3.3
loop two
4246
1.5
loop two
4630
1.1
Clifford Rozier
167
loop two
670
0.7
Clint Capela
8232
loop two
8599
0.2
loop two
9471
0.6
loop two
9722
1.0
loop two
10329
0.9
loop two
10865
1.4
Clyde Drexler
168
loop two
669
5.7
Coby White
10866
Cody Martin
10867
Cody Zeller
7819

4628
0.6
loop two
4925
1.8
loop two
5448
1.2
loop two
5901
2.1
loop two
6433
3.6
loop two
6785
3.2
loop two
7484
2.8
loop two
7811
3.5
loop two
8131
2.1
loop two
8883
1.7
loop two
9474
1.5
David Noel
4629
David Robinson
186
loop two
721
1.3
loop two
942
2.7
loop two
1413
2.1
loop two
1869
1.8
loop two
2449
1.5
loop two
2745
1.2
David Vaughn
187
loop two
728
0.2
loop two
943
0.5
David West
3316
loop two
3771
0.8
loop two
4397
0.8
loop two
4631
1.2
loop two
4926
2.2
loop two
5447
2.3
loop two
5900
2.3
loop two
6434
3.0
loop two
6784
2.3
loop two
7485
2.1
loop two
7827
2.9
loop two
8134
2.8
loop two
8895
3.4
loop two
9476
1.8
loop two
9683
2.2
David Wingate
189
loop two
723
1.2
loop two
1060
0.6
loop two
1349
0.3
loop two
1788
0.4
Davis Bertans
9498
loop two
9684
0.7
loop two
10357
1.0
loop two
10827
1.3
Davon Reed
9685
loop two
10328
0.6
De'Aaron Fox
9686
loop two
10358
4.4
loop two
10828
7.3
De'Andre Hunter
10829
De'Anthony Melton
10327
loop two
10830
3.2
DeAndre Jordan
5446
loop two
58

...

9432
2.2
Pape Sow
3542
loop two
4052
0.1
loop two
4431
0.2
Pape Sy
6623
Pascal Siakam
9426
loop two
9769
0.3
loop two
10457
2.0
loop two
10926
3.1
Pat Connaughton
8711
loop two
9333
0.3
loop two
9767
0.7
loop two
10394
1.1
loop two
10927
2.0
Pat Garrity
1165
loop two
1715
0.5
loop two
2009
0.7
loop two
2249
0.7
loop two
3025
1.2
loop two
3106
1.5
loop two
3541
0.5
loop two
4170
0.4
loop two
4852
0.7
loop two
5220
0.4
Patrick Beverley
7163
loop two
7667
2.9
loop two
8560
2.7
loop two
8712
3.4
loop two
9335
3.4
loop two
9766
4.2
loop two
10395
2.9
loop two
10928
3.8
Patrick Ewing
410
loop two
475
2.0
loop two
1166
1.1
loop two
1716
1.1
loop two
2008
0.9
loop two
2237
1.2
loop two
6622
0.5
Patrick McCaw
9336
loop two
9765
1.1
loop two
10396
1.4
loop two
10929
1.0
Patrick O'Bryant
4853
loop two
5303
0.6
loop two
5662
0.2
loop two
6193
0.3
Patrick Patterson
6621
loop two
6808
0.8
loop two
7162
0.8
loop two
7684
1.1
loop two
8562
1.2
loop two
8713
1.9
loop two
9337
1.2
loop two
9764
1.2
loop

loop two
624
1.9
loop two
1186
0.9
loop two
1571
1.8
Richard Hamilton
1572
loop two
2173
1.5
loop two
2380
2.9
loop two
2888
2.7
loop two
3248
2.5
loop two
3690
4.0
loop two
4145
4.9
loop two
4711
3.4
loop two
5305
3.8
loop two
5620
4.2
loop two
6053
4.4
loop two
6665
4.4
loop two
6889
3.1
loop two
7272
3.0
Richard Jefferson
2379
loop two
2889
1.8
loop two
3247
2.5
loop two
3689
3.8
loop two
4144
4.0
loop two
4712
3.8
loop two
5306
2.7
loop two
5635
3.1
loop two
6054
2.4
loop two
6664
2.0
loop two
6890
1.3
loop two
7256
1.4
loop two
7640
0.6
loop two
8458
1.6
loop two
8686
0.8
loop two
9358
0.8
loop two
9771
1.0
Richard Manning
263
Richaun Holmes
8674
loop two
9359
0.6
loop two
9789
1.0
loop two
10413
1.3
loop two
10943
0.9
Rick Fox
264
loop two
622
3.8
loop two
1156
3.4
loop two
1574
2.0
loop two
2152
1.7
loop two
2377
3.2
loop two
2891
3.5
loop two
3256
3.3
Rick Mahorn
265
loop two
621
0.3
loop two
1155
0.3
Ricky Davis
1139
loop two
1576
1.3
loop two
2151
1.3
loop two
2376
1.6
loop t

2.2
loop two
7299
2.1
loop two
7650
3.3
loop two
8445
3.2
loop two
9001
3.3
loop two
9317
3.0
loop two
9896
1.8
loop two
10376
2.0
Shawn Bradley
242
loop two
647
0.7
loop two
1053
0.9
loop two
1558
0.8
loop two
2186
0.8
loop two
2415
0.5
loop two
2638
0.4
loop two
3266
0.7
loop two
3715
0.3
Shawn Kemp
243
loop two
658
1.9
loop two
1052
2.5
loop two
1559
2.4
loop two
2145
1.7
loop two
2403
1.0
loop two
2868
0.7
Shawn Marion
1560
loop two
2154
1.4
loop two
2401
2.0
loop two
2869
2.0
loop two
3265
2.4
loop two
3714
2.7
loop two
4140
1.9
loop two
4719
1.8
loop two
5244
1.7
loop two
5732
2.2
loop two
6061
2.0
loop two
6567
1.4
loop two
6859
1.4
loop two
7301
2.1
loop two
7648
2.4
loop two
8447
1.6
Shawn Respert
332
loop two
646
1.0
loop two
1051
1.1
Shawne Williams
4748
loop two
5243
0.5
loop two
5749
0.9
loop two
6566
0.1
loop two
6860
0.7
loop two
7644
0.6
loop two
8448
0.8
Shawnelle Scott
245
loop two
644
0.0
loop two
2143
0.2
loop two
2400
0.1
Shelden Williams
4749
loop two
5242
0.5
loo

loop two
560
0.3
loop two
981
0.2
loop two
1638
0.1
loop two
2090
0.1
Todd MacCulloch
1639
loop two
2091
0.2
loop two
2320
0.2
loop two
2946
1.3
Tom Chambers
326
loop two
559
0.3
Tom Gugliotta
327
loop two
558
4.1
loop two
989
4.1
loop two
1640
2.8
loop two
2114
2.3
loop two
2319
1.0
loop two
2947
1.8
loop two
3200
1.1
loop two
3636
1.1
Tom Hammonds
328
loop two
557
0.8
loop two
980
0.6
loop two
1641
0.4
loop two
2115
0.2
Tomas Satoransky
9417
loop two
9912
1.6
loop two
10472
3.9
loop two
11002
5.0
Toney Douglas
6112
loop two
6677
2.0
loop two
6883
3.0
loop two
7285
2.0
loop two
7768
2.1
loop two
8492
1.3
loop two
8972
2.0
loop two
9431
2.6
Toni Kukoc
329
loop two
556
4.5
loop two
978
4.2
loop two
1642
5.3
loop two
2116
4.7
loop two
2318
3.1
loop two
2948
3.5
loop two
3188
3.7
loop two
3635
2.7
loop two
4092
3.0
Tony Allen
3634
loop two
4091
0.8
loop two
4771
1.3
loop two
5275
1.7
loop two
5688
1.5
loop two
6113
1.4
loop two
6578
1.3
loop two
6797
1.4
loop two
7293
1.4
loop two
7770
1.

2.2
loop two
1610
0.8
loop two
2118
2.6
loop two
2347
2.4
loop two
2917
1.8
loop two
3217
2.3
loop two
3668
2.1
loop two
4128
2.0
Wade Baldwin IV
9121
loop two
9846
1.8
loop two
10437
0.7
Walker Russell
6727
Wally Szczerbiak
1611
loop two
1980
2.8
loop two
2345
3.2
loop two
2918
3.1
loop two
3228
2.6
loop two
3667
1.2
loop two
4127
2.4
loop two
4734
3.0
loop two
5259
1.7
loop two
5708
1.4
Walt Williams
292
loop two
588
2.7
loop two
999
2.1
loop two
1612
1.7
loop two
2057
2.1
loop two
2344
1.3
loop two
2919
1.4
Walter McCarty
293
loop two
587
0.4
loop two
998
2.2
loop two
1613
1.3
loop two
1978
1.1
loop two
2343
0.7
loop two
2920
0.7
loop two
3216
1.3
loop two
3666
1.6
loop two
4126
0.5
Walter Sharpe
5706
Wang Zhi-zhi
1868
loop two
2342
0.0
loop two
2921
0.4
loop two
3214
0.2
loop two
3665
0.1
Wayman Tisdale
294
Wayne Ellington
6075
loop two
6590
1.0
loop two
6728
1.2
loop two
7188
0.6
loop two
7722
1.3
loop two
8151
0.4
loop two
9054
1.6
loop two
9122
1.1
loop two
9868
1.1
loop two
104

In [13]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9186 entries, 0 to 11142
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         9186 non-null   int64  
 1   player_name        9186 non-null   object 
 2   Position           8778 non-null   object 
 3   team_abbreviation  9186 non-null   object 
 4   age                9186 non-null   int64  
 5   player_height      9186 non-null   float64
 6   player_weight      9186 non-null   float64
 7   college            9186 non-null   object 
 8   country            9186 non-null   object 
 9   draft_year         9186 non-null   int32  
 10  draft_round        9186 non-null   int32  
 11  draft_number       9186 non-null   int32  
 12  gp                 9186 non-null   int64  
 13  pts                9186 non-null   float64
 14  reb                9186 non-null   float64
 15  ast                9186 non-null   float64
 16  net_rating         9186

In [14]:
player_df["Position"] = player_df["Position"].astype(str)

In [15]:
le = preprocessing.LabelEncoder()
le.fit(player_df["Position"])

LabelEncoder()

In [16]:
player_df["Position"] = le.transform(player_df["Position"])

In [17]:
player_df.head()

,Unnamed: 0,player_name,Position,team_abbreviation,age,player_height,player_weight,college,country,draft_year,...,ts_pct,ast_pct,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
0,0,Dennis Rodman,2,CHI,36,198.12,99.790240,Southeastern Oklahoma State,USA,1986,...,0.479,0.113,1996,10,16.1,5.7,3.1,0.0,0.0,0.0
1,1,Dwayne Schintzius,0,LAC,28,215.90,117.933920,Florida,USA,1990,...,0.430,0.048,1996,6,1.5,2.3,0.3,0.0,0.0,0.0
2,2,Earl Cureton,3,TOR,39,205.74,95.254320,Detroit Mercy,USA,1979,...,0.376,0.148,1996,17,1.0,0.8,0.4,0.0,0.0,0.0
3,3,Ed O'Bannon,2,DAL,24,203.20,100.697424,UCLA,USA,1995,...,0.399,0.077,1996,1,2.3,3.7,0.6,0.0,0.0,0.0
4,4,Ed Pinckney,2,MIA,34,205.74,108.862080,Villanova,USA,1985,...,0.611,0.040,1996,11,2.4,2.4,0.2,0.0,0.0,0.0


In [18]:
player_df = player_df.drop(["team_abbreviation","net_rating","oreb_pct",'dreb_pct','usg_pct','ts_pct',"ast_pct",],axis = 1)

In [19]:
player_df.columns

Index(['Unnamed: 0', 'player_name', 'Position', 'age', 'player_height',
       'player_weight', 'college', 'country', 'draft_year', 'draft_round',
       'draft_number', 'gp', 'pts', 'reb', 'ast', 'season', 'years_played',
       'avg_reb', 'avg_pts', 'avg_ast', 'last_reb', 'last_ast', 'last_pts'],
      dtype='object')

In [20]:
X_Data = player_df[["Position","age","player_height","player_weight","draft_year","draft_round","draft_number","season","years_played","avg_reb","avg_pts","avg_ast",'last_reb','last_ast',"last_pts"]]

In [21]:
X_Data.head()

,Position,age,player_height,player_weight,draft_year,draft_round,draft_number,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
0,2,36,198.12,99.790240,1986,2,27,1996,10,16.1,5.7,3.1,0.0,0.0,0.0
1,0,28,215.90,117.933920,1990,1,24,1996,6,1.5,2.3,0.3,0.0,0.0,0.0
2,3,39,205.74,95.254320,1979,3,58,1996,17,1.0,0.8,0.4,0.0,0.0,0.0
3,2,24,203.20,100.697424,1995,1,9,1996,1,2.3,3.7,0.6,0.0,0.0,0.0
4,2,34,205.74,108.862080,1985,1,10,1996,11,2.4,2.4,0.2,0.0,0.0,0.0


In [22]:
Y_Data = player_df["reb"]

In [23]:
Y_Data.head()

0    16.1
1     1.5
2     1.0
3     2.3
4     2.4
Name: reb, dtype: float64

In [24]:
X_Train , X_Test , Y_Train, Y_test = train_test_split(X_Data , Y_Data, test_size = .3)

In [25]:
model = linear_model.LinearRegression()

In [26]:
player_df.describe()

,Unnamed: 0,Position,age,player_height,player_weight,draft_year,draft_round,draft_number,gp,pts,reb,ast,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
count,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000,9186.000000
mean,5478.422164,3.338341,27.163945,201.466516,101.461409,2002.371108,1.301981,21.721424,55.027651,8.859373,3.843664,1.921043,2007.676246,5.305138,3.913113,9.026983,1.913642,3.413684,1.708448,7.953875
std,3191.376617,2.022208,4.475362,8.978333,12.472153,8.188896,0.509283,16.058054,23.638617,6.100357,2.541939,1.853962,6.932297,4.246792,2.337459,5.405855,1.741604,2.791849,1.899481,6.582172
min,0.000000,0.000000,18.000000,160.020000,60.327736,1963.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1996.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2706.250000,2.000000,24.000000,195.580000,92.986360,1996.000000,1.000000,9.000000,39.000000,4.100000,2.000000,0.600000,2002.000000,2.000000,2.200000,4.871250,0.700000,1.400000,0.300000,2.700000
50%,5480.500000,3.000000,26.500000,203.200000,102.058200,2003.000000,1.000000,19.000000,61.000000,7.500000,3.300000,1.300000,2008.000000,5.000000,3.400000,7.900000,1.350000,3.000000,1.100000,6.900000
75%,8212.500000,5.000000,30.000000,208.280000,111.130040,2009.000000,2.000000,32.000000,76.000000,12.400000,5.100000,2.600000,2014.000000,8.000000,5.110417,12.316071,2.600000,4.900000,2.400000,12.000000
max,11142.000000,7.000000,44.000000,231.140000,155.582056,2019.000000,8.000000,165.000000,85.000000,36.100000,16.300000,11.700000,2019.000000,48.000000,16.100000,29.600000,11.400000,16.300000,11.700000,36.100000


In [27]:
model.fit(X_Train , Y_Train)

LinearRegression()

In [28]:
model.coef_

array([ 2.08467212e-02, -1.10906041e-01,  8.29478474e-03, -7.42817819e-03,
       -1.40319274e+10, -1.58752929e-01,  1.24536390e-02,  1.40319274e+10,
       -1.40319274e+10,  9.74732180e-01, -2.45962604e-02,  3.64071095e-02,
        7.76711607e-02, -5.75456495e-03,  3.23690625e-02])

In [29]:
a=model.predict(X_Test)

In [30]:
a[1]

3.252546402355062

In [31]:
Y_test

1856     2.8
7033     5.4
6469     8.3
4260     8.0
3599     1.6
        ... 
1758     3.6
8179     3.8
10717    6.9
8224     7.0
1954     2.2
Name: reb, Length: 2756, dtype: float64

In [32]:
np.mean((a-Y_test)**2)

1.057446774213335

In [33]:
#test_df = player_df.loc[(player_df["player_name"] == "LeBron James") & (player_df["season"] == 2019)]
#test_df.head(15)

,Unnamed: 0,player_name,Position,age,player_height,player_weight,college,country,draft_year,draft_round,...,reb,ast,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
11042,11042,LeBron James,4,35,205.74,113.398,None,USA,2003,1,...,7.8,10.7,2019,16,7.447059,27.064706,7.441176,8.5,8.3,27.4


In [34]:
new_season_df = player_df.loc[player_df["season"] == 2019]
name_key = new_season_df["player_name"]
#new_season_df["last_reb"] = new_season_df["reb"]
#new_season_df["last_pts"] = new_season_df["pts"]
#new_season_df["last_ast"] = new_season_df["ast"]
new_season_df.head()

<ipython-input-34-f51e0d1edbe9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_season_df["last_reb"] = new_season_df["reb"]
<ipython-input-34-f51e0d1edbe9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_season_df["last_pts"] = new_season_df["pts"]
<ipython-input-34-f51e0d1edbe9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,Unnamed: 0,player_name,Position,age,player_height,player_weight,college,country,draft_year,draft_round,...,reb,ast,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
10631,10631,Georges Niang,2,26,200.66,104.326160,Iowa State,USA,2016,2,...,1.9,0.6,2019,3,1.275000,2.925000,0.425000,1.9,0.6,5.8
10632,10632,George Hill,5,33,190.50,85.275296,None,USA,2008,1,...,3.0,2.9,2019,11,3.125000,11.350000,3.258333,3.0,2.9,9.7
10635,10635,Gary Harris,5,25,193.04,95.254320,Michigan State,USA,2014,1,...,2.9,2.1,2019,5,2.583333,11.883333,2.083333,2.9,2.1,10.3
10638,10638,Giannis Antetokounmpo,4,25,210.82,109.769264,None,Greece,2013,1,...,13.7,5.8,2019,6,9.114286,20.500000,4.385714,13.7,5.8,29.6
10640,10640,Furkan Korkmaz,5,22,200.66,91.625584,None,Turkey,2016,1,...,2.3,1.1,2019,3,1.766667,5.666667,0.833333,2.3,1.1,9.6


In [35]:
#new_season_df["age"] = new_season_df["age"]+1

<ipython-input-35-7ef4508fc08c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_season_df["age"] = new_season_df["age"]+1


In [36]:
#new_season_df["years_played"] = new_season_df["years_played"]+1

<ipython-input-36-a3ca3130c9c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_season_df["years_played"] = new_season_df["years_played"]+1


In [37]:
#new_season_df["season"] = new_season_df["season"]+1

<ipython-input-37-1d197d8d8cb6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_season_df["season"] = new_season_df["season"]+1


In [38]:
new_season_df = new_season_df[["Position","age","player_height","player_weight","draft_year","draft_round","draft_number","season","years_played","avg_reb","avg_pts","avg_ast",'last_reb','last_ast',"last_pts"]]

In [39]:
new_season_df.head()

,Position,age,player_height,player_weight,draft_year,draft_round,draft_number,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
10631,2,27,200.66,104.326160,2016,2,50,2020,4,1.275000,2.925000,0.425000,1.9,0.6,5.8
10632,5,34,190.50,85.275296,2008,1,26,2020,12,3.125000,11.350000,3.258333,3.0,2.9,9.7
10635,5,26,193.04,95.254320,2014,1,19,2020,6,2.583333,11.883333,2.083333,2.9,2.1,10.3
10638,4,26,210.82,109.769264,2013,1,15,2020,7,9.114286,20.500000,4.385714,13.7,5.8,29.6
10640,5,23,200.66,91.625584,2016,1,26,2020,4,1.766667,5.666667,0.833333,2.3,1.1,9.6


In [40]:
#test_df = test_df[["Position","age","player_height","player_weight","draft_year","draft_round","draft_number","season","years_played","avg_reb","avg_pts","avg_ast",'last_reb','last_ast',"last_pts"]]
#test_df.head()

,Position,age,player_height,player_weight,draft_year,draft_round,draft_number,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts
11042,4,35,205.74,113.398,2003,1,1,2019,16,7.447059,27.064706,7.441176,8.5,8.3,27.4


In [41]:
a = model.predict(new_season_df)

In [42]:
model.coef_

array([ 2.08467212e-02, -1.10906041e-01,  8.29478474e-03, -7.42817819e-03,
       -1.40319274e+10, -1.58752929e-01,  1.24536390e-02,  1.40319274e+10,
       -1.40319274e+10,  9.74732180e-01, -2.45962604e-02,  3.64071095e-02,
        7.76711607e-02, -5.75456495e-03,  3.23690625e-02])

In [43]:
a

array([ 1.23719413,  2.26674063,  2.50155532, 10.14296283,  2.27600006,
        2.53272891,  3.82874791,  2.42434838,  2.40319803,  5.49407308,
        3.07504627,  1.55109516,  3.26872152,  4.5572674 ,  4.54760243,
        9.49228265,  2.07192796,  3.14251725,  1.04042521,  2.56253248,
        2.43405078,  1.14943438,  2.64187101,  2.05825963,  4.40087418,
        4.06215243,  1.59353065,  3.92990979,  6.41057451,  2.45956044,
       10.43002158,  4.33452099,  4.01008331,  1.98360916,  5.41867236,
        1.42979806,  4.45899974,  2.02092479,  2.77728535,  7.12968179,
        3.44111043,  1.43381183,  4.97907105,  4.79571182,  3.01393336,
        3.22082612,  1.5419434 ,  3.46193215,  5.58637806,  1.72316757,
        1.37689117,  6.65754443,  2.9293676 ,  1.76147672,  3.93804367,
        3.35952338,  1.65764317,  1.74466318,  4.92681924,  4.99579421,
        2.51608237, 11.26211982,  9.87524295,  4.53299067,  2.49498872,
        8.72048605,  5.83191434,  2.98737553,  0.9227021 ,  3.15

In [44]:
Pts_X_Data = player_df[["Position","age","player_height","player_weight","draft_year","draft_round","draft_number","season","years_played","avg_reb","avg_pts","avg_ast",'last_reb','last_ast',"last_pts"]]

In [45]:
Pts_Y_Data = player_df["pts"]

In [46]:
X_Train_pts , X_Test_pts , Y_Train_pts, Y_test_pts = train_test_split(Pts_X_Data , Pts_Y_Data, test_size = .3)

In [47]:
model_pts = linear_model.LinearRegression()

In [48]:
model_pts.fit(X_Train_pts,Y_Train_pts)

LinearRegression()

In [49]:
model_pts.coef_

array([ 3.52638816e-02, -2.87466770e-01,  1.70454442e-02, -1.82137842e-02,
        4.75830773e+09, -4.20888355e-01,  3.92398285e-02, -4.75830773e+09,
        4.75830773e+09,  2.06144314e-01,  8.17117619e-01,  7.15007777e-02,
       -9.73827572e-02, -1.72724800e-02,  2.43887987e-01])

In [50]:
b=model_pts.predict(new_season_df)

In [51]:
b

array([ 3.33127413,  8.88453115, 11.53073787, 23.46071008,  7.54807058,
        6.51091213, 12.967553  , 12.81226885,  6.57765266,  6.17894298,
        4.61481914,  6.9558783 ,  6.90042819, 13.17038255,  8.42244084,
       10.55978187,  2.75822441,  2.37180382,  3.23376711,  4.60041032,
        4.3141687 ,  3.64435214, 14.73611561,  7.63997233, 24.57704136,
        8.23120133,  6.82505663,  5.22337308,  7.73153229,  5.91515307,
       11.90527343,  7.31242384,  9.64564018,  4.93327819,  3.2024046 ,
        4.24331378, 10.58541741,  5.60657121, 10.14888353,  9.54325436,
       13.10868586, 14.09605899, 15.50194956,  7.41791224,  5.01829547,
        5.50788057,  6.88461496,  4.94922404,  8.07097073, 15.17664486,
        6.16040045, 20.27780546,  8.67007715, 10.37749739, 10.17506733,
        9.58736687,  3.73492413,  4.52012984, 16.87991909, 15.44991594,
        8.53304403, 24.0997897 , 20.08409896,  5.48848641,  3.03777557,
       11.88701164, 10.27246699,  3.69179454,  2.80350219, 15.37

In [52]:
Y_test_pts

5475      9.6
155      16.2
2094      1.5
1960      7.1
1341      4.8
         ... 
6244     23.5
4041     10.1
11081     0.0
8678     10.3
6070      9.4
Name: pts, Length: 2756, dtype: float64

In [53]:
Ast_X_Data = player_df[["Position","age","player_height","player_weight","draft_year","draft_round","draft_number","season","years_played","avg_reb","avg_pts","avg_ast",'last_reb','last_ast',"last_pts"]]

In [54]:
Ast_Y_Data = player_df["ast"]

In [55]:
X_Train_ast , X_Test_ast , Y_Train_ast, Y_test_ast = train_test_split(Ast_X_Data , Ast_Y_Data, test_size = .3)

In [56]:
model_ast = linear_model.LinearRegression()

In [57]:
model_ast.fit(X_Train_ast,Y_Train_ast)

LinearRegression()

In [58]:
c = model_ast.predict(new_season_df)

In [59]:
c

array([ 0.53157794,  2.84506213,  2.07540117,  5.09688786,  1.2230905 ,
        0.98491447,  3.04022612,  4.63565779,  3.05807586,  1.14460409,
        1.23705556,  1.16611837,  0.86095301,  1.67540182,  1.73363343,
        0.6343965 ,  0.40881481,  0.22371181,  0.66901226,  1.33783104,
        0.66074379,  1.80328875,  2.77966368,  1.10101635,  4.38602   ,
        1.96355965,  0.64577926,  1.32770311,  5.02388419,  1.16714626,
        0.75403908,  1.05866293,  1.71981378,  0.95043378,  0.33727431,
        0.95447552,  6.65685567,  2.33818022,  3.50703367,  0.63224309,
        4.66070618,  2.36091435,  2.36709007,  0.72506729,  1.3284018 ,
        2.65576499,  1.56416994,  1.02963452,  1.15881211,  4.33198171,
        1.52595157,  2.78123532,  0.84614709,  5.38513666,  1.32792624,
        1.50504223,  0.94279819,  1.52813837,  3.94348223,  1.88536955,
        1.4134503 ,  3.32591343,  2.27432777,  0.88732919,  0.46998075,
        1.02519805,  1.36408639,  1.15068227,  1.10738271,  2.74

In [60]:
print(a)
print(b)
print(c)

[ 1.23719413  2.26674063  2.50155532 10.14296283  2.27600006  2.53272891
  3.82874791  2.42434838  2.40319803  5.49407308  3.07504627  1.55109516
  3.26872152  4.5572674   4.54760243  9.49228265  2.07192796  3.14251725
  1.04042521  2.56253248  2.43405078  1.14943438  2.64187101  2.05825963
  4.40087418  4.06215243  1.59353065  3.92990979  6.41057451  2.45956044
 10.43002158  4.33452099  4.01008331  1.98360916  5.41867236  1.42979806
  4.45899974  2.02092479  2.77728535  7.12968179  3.44111043  1.43381183
  4.97907105  4.79571182  3.01393336  3.22082612  1.5419434   3.46193215
  5.58637806  1.72316757  1.37689117  6.65754443  2.9293676   1.76147672
  3.93804367  3.35952338  1.65764317  1.74466318  4.92681924  4.99579421
  2.51608237 11.26211982  9.87524295  4.53299067  2.49498872  8.72048605
  5.83191434  2.98737553  0.9227021   3.15962269  4.67745053  9.15742381
  0.79059972  4.21360969  3.8734469   5.66275816  1.4529184   3.81601136
  4.40905086  1.8125804   4.61385629  2.59874685  4

In [63]:
output_df = new_season_df
output_df["player_name"] = name_key

In [88]:
output_df.head()

,Position,age,player_height,player_weight,draft_year,draft_round,draft_number,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts,player_name
10631,2,27,200.66,104.326160,2016,2,50,2020,4,1.275000,2.925000,0.425000,1.9,0.6,5.8,Georges Niang
10632,5,34,190.50,85.275296,2008,1,26,2020,12,3.125000,11.350000,3.258333,3.0,2.9,9.7,George Hill
10635,5,26,193.04,95.254320,2014,1,19,2020,6,2.583333,11.883333,2.083333,2.9,2.1,10.3,Gary Harris
10638,4,26,210.82,109.769264,2013,1,15,2020,7,9.114286,20.500000,4.385714,13.7,5.8,29.6,Giannis Antetokounmpo
10640,5,23,200.66,91.625584,2016,1,26,2020,4,1.766667,5.666667,0.833333,2.3,1.1,9.6,Furkan Korkmaz


In [92]:
output_df["pred_reb"] = 0
output_df["pred_pts"] = 0
output_df["pred_ast"] = 0
output_df.head()

,Position,age,player_height,player_weight,draft_year,draft_round,draft_number,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts,player_name,pred_reb,pred_pts,pred_ast
10631,2,27,200.66,104.326160,2016,2,50,2020,4,1.275000,2.925000,0.425000,1.9,0.6,5.8,Georges Niang,0,0,0
10632,5,34,190.50,85.275296,2008,1,26,2020,12,3.125000,11.350000,3.258333,3.0,2.9,9.7,George Hill,0,0,0
10635,5,26,193.04,95.254320,2014,1,19,2020,6,2.583333,11.883333,2.083333,2.9,2.1,10.3,Gary Harris,0,0,0
10638,4,26,210.82,109.769264,2013,1,15,2020,7,9.114286,20.500000,4.385714,13.7,5.8,29.6,Giannis Antetokounmpo,0,0,0
10640,5,23,200.66,91.625584,2016,1,26,2020,4,1.766667,5.666667,0.833333,2.3,1.1,9.6,Furkan Korkmaz,0,0,0


In [103]:
count = 0
for prediction in a:
    output_df["pred_reb"][count:count+1] = prediction
    count = count + 1

<ipython-input-103-969babb2834d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df["pred_reb"][count:count+1] = prediction


In [104]:
count = 0
for prediction in b:
    output_df["pred_pts"][count:count+1] = prediction
    count = count + 1

<ipython-input-104-3f1d5aace091>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df["pred_pts"][count:count+1] = prediction


In [106]:
count = 0
for prediction in c:
    output_df["pred_ast"][count:count+1] = prediction
    count = count + 1

<ipython-input-106-dfd83a170bb7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df["pred_ast"][count:count+1] = prediction


In [107]:
output_df.head()

,Position,age,player_height,player_weight,draft_year,draft_round,draft_number,season,years_played,avg_reb,avg_pts,avg_ast,last_reb,last_ast,last_pts,player_name,pred_reb,pred_pts,pred_ast
10631,2,27,200.66,104.326160,2016,2,50,2020,4,1.275000,2.925000,0.425000,1.9,0.6,5.8,Georges Niang,1.237194,3.331274,0.531578
10632,5,34,190.50,85.275296,2008,1,26,2020,12,3.125000,11.350000,3.258333,3.0,2.9,9.7,George Hill,2.266741,8.884531,2.845062
10635,5,26,193.04,95.254320,2014,1,19,2020,6,2.583333,11.883333,2.083333,2.9,2.1,10.3,Gary Harris,2.501555,11.530738,2.075401
10638,4,26,210.82,109.769264,2013,1,15,2020,7,9.114286,20.500000,4.385714,13.7,5.8,29.6,Giannis Antetokounmpo,10.142963,23.460710,5.096888
10640,5,23,200.66,91.625584,2016,1,26,2020,4,1.766667,5.666667,0.833333,2.3,1.1,9.6,Furkan Korkmaz,2.276000,7.548071,1.223090


In [111]:
output_df.to_csv("output.csv")